https://towardsdatascience.com/build-your-own-transformer-from-scratch-using-pytorch-84c850470dcb

https://pytorch.org/docs/stable/generated/torch.nn.Transformer.html

https://github.com/pytorch/examples/blob/main/word_language_model/model.py

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import math
import copy
import numpy as np
import random

In [2]:
from transformer import TransformerModel

In [3]:
from net import Net_CBOW
import numpy as np
version = "april27_WT2_nodatalim_10epoch_128dim_100minf"

vocab = torch.load(f"saves/vocab_{version}.pt")
len(vocab)

2156

In [4]:
def lookup_id(word, vocab=vocab):
    if word not in vocab:
        return vocab["<unk>"]
    return vocab[word]
def lookup_token(word_id, vocab=vocab):
    for word in vocab:
        if vocab[word] == word_id:
            return word
    return None

In [71]:
from datasets import load_dataset
wikitext2 = load_dataset("wikitext", "wikitext-2-v1")
text_train = wikitext2["train"]['text']
text_train = [item.lower().strip() for item in text_train if len(item) > 0]
text_test = wikitext2["test"]['text']
text_test = [item.lower().strip() for item in text_test if len(item) > 0]
len(text_test)
text_train = [item.split(" ") + ["\n"] for item in text_train if "=" not in item]
text_test = [item.split(" ") + ["\n"] for item in text_test if "=" not in item]

seq_length = 128
buffer = 20

x_train = [[lookup_id(word) for word in paragraph[len(paragraph)-seq_length-1:len(paragraph)-1]] for paragraph in text_train if len(paragraph) >= seq_length + buffer+1]
y_train = [[lookup_id(word) for word in paragraph[len(paragraph)-seq_length:len(paragraph)]] for paragraph in text_train if len(paragraph) >= seq_length + buffer+1]
x_test = [[lookup_id(word) for word in paragraph[len(paragraph)-seq_length-1:len(paragraph)-1]] for paragraph in text_test if len(paragraph) >= seq_length + buffer+1]
y_test = [[lookup_id(word) for word in paragraph[len(paragraph)-seq_length:len(paragraph)]] for paragraph in text_test if len(paragraph) >= seq_length + buffer+1]
# print([[word for word in paragraph[start_i:max_seq_length]] for paragraph in text_train if len(paragraph) >= max_seq_length + start_i])

In [72]:
print([lookup_token(i) for i in x_train[0]])
print([lookup_token(i) for i in y_train[0]])

['<unk>', 'as', 'they', 'are', '<unk>', '.', 'the', 'route', 'to', 'each', 'story', 'location', 'on', 'the', '<unk>', '<unk>', '<unk>', 'on', 'an', 'individual', 'player', "'s", 'approach', ':', 'when', 'one', '<unk>', 'is', 'selected', ',', 'the', 'other', 'is', '<unk>', 'off', 'to', 'the', 'player', '.', 'outside', '<unk>', ',', 'the', 'player', 'characters', 'rest', 'in', 'a', 'camp', ',', 'where', 'units', 'can', 'be', '<unk>', 'and', 'character', 'growth', '<unk>', '.', 'alongside', 'the', 'main', 'story', '<unk>', 'are', 'character', '@-@', 'specific', '<unk>', '<unk>', '<unk>', 'to', 'different', '<unk>', 'members', '.', 'after', 'the', 'game', "'s", '<unk>', ',', 'additional', 'episodes', 'are', '<unk>', ',', 'some', 'of', 'them', 'having', 'a', 'higher', '<unk>', 'than', 'those', 'found', 'in', 'the', 'rest', 'of', 'the', 'game', '.', 'there', 'are', 'also', 'love', '<unk>', 'elements', 'related', 'to', 'the', 'game', "'s", 'two', 'main', '<unk>', ',', 'although', 'they', 'tak

In [76]:
x_train = x_train
y_train = y_train
x_test = x_test[:100]
x_test = x_test[:100]

In [77]:
v_size = len(vocab)
d_model = 512
num_heads = 8
num_layers = 6
d_feedfoward = 2048
dropout = 0.5

transformer = TransformerModel(ntoken = v_size, ninp = d_model, nhead = num_heads, nhid = d_feedfoward, nlayers = num_layers, dropout = dropout)

print(len(x_train))
x_train = x_train
y_train = y_train
src_data = torch.tensor(x_train)

unk_id = lookup_id("<unk>")
count_unk = sum(sum((i == unk_id) for i in paragraph) for paragraph in src_data).item()
count_total = sum(sum(1 for i in paragraph) for paragraph in src_data)
print(count_unk, count_total, count_unk/count_total)

# a = torch.randint(1, src_vocab_size, (64, max_seq_length))
# print(a[:, 1:])
# print(a[:, :-1])
len(x_train)


5245
149417 671360 0.2225586868446139


5245

In [82]:
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

transformer.train()

batch_size = 20
num_batches = len(x_train) // batch_size
print("Batches per epoch:", num_batches)

percent_data_per_epoch = 1

indices = list(range(len(x_train)))
for epoch in range(20):
    train_loss = 0
    x_train_copy = [x_train[indices[i]] for i in range(len(indices))]
    y_train_copy = [y_train[indices[i]] for i in range(len(indices))]
    for batch in range(int(num_batches * percent_data_per_epoch)):
        if batch == 400:
            print("^^^")
        source = torch.tensor(x_train_copy[batch*batch_size:(batch+1)*batch_size])  # (batch_size, seq_length)
        target = torch.tensor(y_train_copy[batch*batch_size:(batch+1)*batch_size])  # (batch_size, seq_length)
        optimizer.zero_grad()
        output = transformer(source)
        output = output.view(-1, v_size)
        loss = criterion(output, target.view(-1))
        # print(output.shape)
        # print(target.view(-1).shape)
        # print("________")
        # print(loss)
        loss.backward()
        optimizer.step()
        print("|", end="")
        train_loss += loss.item()
    print(" -> ", end='')
    #eval
    test_loss = 0
    count_loss = 0
    with torch.no_grad():
        for datax, datay in zip(x_test,y_test):
            source = torch.tensor([datax])
            target = torch.tensor([datay]).view(-1)
            output = transformer(source)
            output = output.view(-1, v_size)
            test_loss += criterion(output, target).item() * len(datax)
            count_loss += len(datax)
    test_loss /= count_loss

    train_loss /= int(num_batches * percent_data_per_epoch)
    random.shuffle(indices)
    print(f"Epoch: {epoch+1}, Train Loss: {train_loss}, Test Loss: {test_loss}")

torch.save(transformer, "saves/model_transformer_apr30_0230pm.pt")

Batches per epoch: 5245
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||^^^
|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||

KeyboardInterrupt: 

In [ ]:
# torch.save(transformer, "saves/model_transformer_apr29_1200pm.pt")

In [ ]:
o = transformer(src_data[0:1])
sm = np.array(torch.softmax(o, 1)[0].detach())
ids = [list(v).index(max(v)) for v in sm]
words = [lookup_token(i) for i in ids]
print([lookup_token(i) for i in src_data[0]])
print(words)

['control', 'of', 'a', 'military', 'unit', 'and', 'take', 'part', 'in', '<unk>', 'against', 'enemy', 'forces', '.', 'stories', 'are', 'told', 'through', 'comic', 'book', '@-@', 'like', '<unk>', 'with', '<unk>', 'character', '<unk>', ',', 'with', 'characters', '<unk>', '<unk>', 'through', '<unk>', '<unk>', '<unk>', 'and', '<unk>', 'through', '<unk>', 'text', '.', 'the', 'player', '<unk>', 'through', 'a', 'series', 'of', '<unk>', '<unk>', ',', 'gradually', '<unk>', 'as', '<unk>', 'that', 'can', 'be', '<unk>', '<unk>', 'through', 'and', '<unk>', 'as', 'they', 'are', '<unk>', '.', 'the', 'route', 'to', 'each', 'story', 'location', 'on', 'the', '<unk>', '<unk>', '<unk>', 'on', 'an', 'individual', 'player', "'s", 'approach', ':', 'when', 'one', '<unk>', 'is', 'selected', ',', 'the', 'other', 'is', '<unk>', 'off', 'to', 'the', 'player', '.', 'outside', '<unk>', ',', 'the', 'player', 'characters', 'rest', 'in', 'a', 'camp', ',', 'where', 'units', 'can', 'be', '<unk>', 'and', 'character', 'grow

In [ ]:
text = "you are a helpful assistant . Answer the following question . "

text = [lookup_id(word.lower()) for word in text.strip().split(" ")]

for count in range(50):
    i = torch.tensor([text])
    o = transformer(i, i)
    sm = np.array(torch.softmax(o, 2)[0].detach())
    top5 = [np.zeros(5) for _ in range(len(sm))]
    top5p = [np.zeros(5) for _ in range(len(sm))]
    for vi in range(len(sm)-1,len(sm)):
        v = sm[vi]
        for item in v:
            m = top5[vi][list(top5[vi]).index(min(top5[vi]))]
            if lookup_token(list(v).index(item)) != "<unk>":
                top5[vi][list(top5[vi]).index(min(top5[vi]))] = max(m, item)
        top5[vi] = [list(v).index(i) for i in top5[vi]]
        top5p[vi] = [v[i] for i in top5[vi]]
    # ids = [list(v).index(max(v)) for v in sm]
    words = [[lookup_token(i) for i in w] for w in top5][-1]
    # print([lookup_token(i) for i in src_data[0]])
    # print(words)
    print(' '.join([lookup_token(i) for i in text]))
    print(words)
    print(top5p[-1])
    chosen_word = words[int(input("Choose a word: "))-1]
    text.append(lookup_id(chosen_word))
# text


you are a <unk> <unk> . <unk> the following <unk> .
['after', 'it', 'are', 'a', 'this']
[0.008957216, 0.013795887, 0.01188761, 0.11148329, 0.016574614]
you are a <unk> <unk> . <unk> the following <unk> . a
['a', 'few', 'large', 'small', '"']
[0.014941752, 0.004584193, 0.005560547, 0.005065698, 0.0033910982]
you are a <unk> <unk> . <unk> the following <unk> . a few
['can', 'a', '.', 'are', 'is']
[0.0043566437, 0.009275039, 0.0067396043, 0.048805617, 0.008312006]
you are a <unk> <unk> . <unk> the following <unk> . a few can
['have', 'are', 'do', 'a', 'be']
[0.057717957, 0.035070132, 0.0101899505, 0.023004128, 0.12751704]
you are a <unk> <unk> . <unk> the following <unk> . a few can have
['a', 'been', 'also', 'are', 'have']
[0.15154977, 0.12867396, 0.006671486, 0.024408525, 0.011477391]
you are a <unk> <unk> . <unk> the following <unk> . a few can have a
['small', 'a', 'large', 'are', 'few']
[0.012695176, 0.0087218555, 0.007897064, 0.006415769, 0.008232827]
you are a <unk> <unk> . <unk> t

ValueError: invalid literal for int() with base 10: ''

In [ ]:
device = torch.device("cpu")
ntokens = len(vocab)
input = torch.randint(ntokens, (1, 1), dtype=torch.long).to(device)
model = transformer
temperature = 2
log_interval = 1
input = 'alongside the main story <unk>'
input = [lookup_id(i) for i in input.strip().split(" ")]
input = torch.tensor(input).view(len(input), 1)


with open('out_generation.txt', 'w') as outf:
    with torch.no_grad():  # no tracking history
        for i in range(20):
            output = model(input, False)
            word_weights = output[-1].squeeze().div(temperature).exp().cpu()
            word_idx = torch.multinomial(word_weights, 1)[0]
            word_tensor = torch.Tensor([[word_idx]]).long().to(device)
            print(word_tensor)
            input = torch.cat([input, word_tensor], 0)

            word = lookup_token(word_idx)

            outf.write(word + ('\n' if i % 20 == 19 else ' '))

            if i % log_interval == 0:
                print('| Generated {}/{} words'.format(i, [lookup_token(i[0])for i in input]))

tensor([[1220]])
| Generated 0/['alongside', 'the', 'main', 'story', '<unk>', 'terms'] words
tensor([[1074]])
| Generated 1/['alongside', 'the', 'main', 'story', '<unk>', 'terms', 'location'] words
tensor([[4]])
| Generated 2/['alongside', 'the', 'main', 'story', '<unk>', 'terms', 'location', '<unk>'] words
tensor([[4]])
| Generated 3/['alongside', 'the', 'main', 'story', '<unk>', 'terms', 'location', '<unk>', '<unk>'] words
tensor([[489]])
| Generated 4/['alongside', 'the', 'main', 'story', '<unk>', 'terms', 'location', '<unk>', '<unk>', 'upon'] words
tensor([[1216]])
| Generated 5/['alongside', 'the', 'main', 'story', '<unk>', 'terms', 'location', '<unk>', '<unk>', 'upon', 'spanish'] words
tensor([[180]])
| Generated 6/['alongside', 'the', 'main', 'story', '<unk>', 'terms', 'location', '<unk>', '<unk>', 'upon', 'spanish', 'you'] words
tensor([[1251]])
| Generated 7/['alongside', 'the', 'main', 'story', '<unk>', 'terms', 'location', '<unk>', '<unk>', 'upon', 'spanish', 'you', 'individ